In [ ]:
import numpy as np
import cv2
import open3d as o3d
import plotly.graph_objects as go
import matplotlib.pyplot as plt
from pathlib import Path

%load_ext autoreload
%autoreload 2
from lac.perception.depth import render_o3d, get_renderer, get_plotly_mesh, get_light_direction, map_to_mesh
from lac.utils.frames import get_cam_pose_rover, CAMERA_TO_OPENCV_PASSIVE

from lac.util import load_data

In [ ]:
data_path = Path("/home/shared/data_raw/LAC/runs/full_spiral_map1_preset1_recovery_agent")
initial_pose, lander_pose, poses, imu_data, cam_config = load_data(data_path)

map_gt = np.load(
    "/home/shared/data_raw/LAC/heightmaps/competition/Moon_Map_01_preset_1.dat",
    allow_pickle=True,
)

In [ ]:
renderer, material = get_renderer()
surface_mesh = map_to_mesh(map_gt)

# Direction that it comes from
el_sun = np.deg2rad(20)
az_sun = np.arctan2(0.0, -1.0)
d_light = get_light_direction(az_sun, el_sun)

In [ ]:
from lac.utils.frames import cam_to_world

frame = 1000
img0 = cv2.imread((str(data_path / "FrontLeft" / f"{frame:06d}.png")))
img1 = cv2.imread((str(data_path / "FrontRight" / f"{frame:06d}.png")))

fig, ax = plt.subplots(1, 2, figsize=(10, 5))
ax[0].imshow(img0)
ax[1].imshow(img1)
plt.show()

cam_T_world = cam_to_world(poses[frame], "FrontLeft")
img_gt, depth_gt = render_o3d([surface_mesh], renderer, material, cam_T_world, d_light)

fig, ax = plt.subplots(1, 2, figsize=(10, 5))
ax[0].imshow(img_gt)
im = ax[1].imshow(depth_gt, cmap="viridis")
plt.colorbar(im, ax=ax[1], orientation="vertical", fraction=0.03)
plt.tight_layout()
plt.show()

In [ ]:
from lac.perception.depth import DepthEstimator
from lac.params import FL_X, STEREO_BASELINE

depth_estimator = DepthEstimator()

disp = depth_estimator.compute_disparity(img0, img1)

depth = (FL_X * STEREO_BASELINE) / (disp + 1e-8)
fig, ax = plt.subplots(1, 2, figsize=(10, 5))
im = ax[0].imshow(depth, cmap="viridis")
plt.colorbar(im, ax=ax[0], fraction=0.03)
error = np.abs(depth - depth_gt)
im = ax[1].imshow(error, cmap="viridis")
plt.colorbar(im, ax=ax[1], fraction=0.03)
plt.tight_layout()
plt.show()

im = plt.imshow(np.where(error < 0.05, error, np.nan), cmap="jet")
plt.colorbar(im, fraction=0.03)
plt.show()

plt.scatter(depth.flatten(), error.flatten(), s=0.5, alpha=0.5, marker="o")
plt.show()

In [ ]:
fig = go.Figure(data=[get_plotly_mesh(m) for m in [surface_mesh]])
fig.update_layout(scene=dict(aspectmode="data"), width=800, height=400, showlegend=False)
from lac.utils.plotting import plot_poses

plot_poses([poses[frame]], fig=fig)
plot_poses(poses, fig=fig, no_axes=True)
fig.show()